In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as optimization

In [3]:
data = pd.read_csv('Uber-2015.csv')
len(data)

14270479

In [2]:
Prediction = pd.read_csv('Prediction_2015.csv')

In [19]:
Prediction.head()

,Unnamed: 0,Location_ID,Uber_Popularity,Uber_a,Uber_b,Uber_c,Uber_d,Taxi_Popularity,taxi_a,taxi_b,taxi_c,taxi_d,Prediction,Prediction_Type,Transformed_ID
0,0,1,Uber_not_popuplar,0.000000,0.000000,0.000000,0.000000,popular,1701.463999,-0.103569,16.408720,236.862157,Taxi_Dominates,Taxi Dominates,82
1,1,2,Uber_not_popuplar,0.000000,0.000000,0.000000,0.000000,Taxi_not_popuplar,0.000000,0.000000,0.000000,0.000000,Not_Popular,Low Trips,151
2,2,3,popular,10.167656,0.112903,4.818614,-1.116316,Taxi_not_popuplar,0.000000,0.000000,0.000000,0.000000,Uber_Dominates,Uber Dominates,172
3,3,4,popular,395.943848,1.077022,141.060494,2.900637,popular,1106.532458,-0.670889,478.565018,-122.301229,225.5358995625786,Expected Uber Dominates,163
4,4,5,Uber_not_popuplar,0.000000,0.000000,0.000000,0.000000,Taxi_not_popuplar,0.000000,0.000000,0.000000,0.000000,Not_Popular,Low Trips,140


In [4]:
Location_transform = np.load('nyc_lookup.npy')

def ID_Transform(id_value, trans = Location_transform):
    index = trans[id_value-1][-1]
    return index

result = map(ID_Transform, Prediction.loc[:, 'Location_ID'].tolist())

Prediction['Transformed_ID'] = result

In [14]:
data = Prediction.groupby('Transformed_ID')['Prediction_Type']

In [20]:
tran = np.unique(Prediction['Transformed_ID'].values)
result = np.zeros((len(tran), 2), dtype = object)
i = 0
for value in tran:
    temp = Prediction[Prediction['Transformed_ID'] == value]
    result[i, 0] = value
    result[i, 1] = temp['Prediction_Type'].tolist()[0]
    i+=1
    

In [25]:
Prediction = pd.DataFrame(result, columns={'Location_ID_new', 'Prediction'})

In [26]:
Prediction.to_csv('Prediction_Final.csv')

In [30]:
filter(lambda x: x[1] == 161, Location_transform)

[array([160, 161, 'Manhattan', 'Midtown Center', 40.75536760000001,
        -73.9784137, 33], dtype=object)]

In [10]:
#data_taxi = pd.read_csv('Taxi_Data.csv')
data_uber = pd.read_csv('Uber-2014.csv')

In [3]:
data_uber.head()

,Unnamed: 0,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID,Date,Year,Month,time
0,0,B02617,2015-05-17 09:47:00,B02617,141,17,2015,5,9
1,1,B02617,2015-05-17 09:47:00,B02617,65,17,2015,5,9
2,2,B02617,2015-05-17 09:47:00,B02617,100,17,2015,5,9
3,3,B02617,2015-05-17 09:47:00,B02774,80,17,2015,5,9
4,4,B02617,2015-05-17 09:47:00,B02617,90,17,2015,5,9


In [11]:
#data_taxi = data_taxi.drop(['Unnamed: 0'], axis = 1)
data_uber = data_uber.drop(['Unnamed: 0'], axis = 1)

In [5]:
data_uber.head()

,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID,Date,Year,Month,time
0,B02617,2015-05-17 09:47:00,B02617,141,17,2015,5,9
1,B02617,2015-05-17 09:47:00,B02617,65,17,2015,5,9
2,B02617,2015-05-17 09:47:00,B02617,100,17,2015,5,9
3,B02617,2015-05-17 09:47:00,B02774,80,17,2015,5,9
4,B02617,2015-05-17 09:47:00,B02617,90,17,2015,5,9


In [28]:
uber_ID_list = np.unique(data_uber['Location_ID'].values)

In [12]:
data_uber = data_uber.groupby(['Location_ID'])['time'].count().reset_index(level = 1)

In [14]:
Location_transform = np.load('nyc_lookup.npy')

In [15]:
Location_transform = np.load('nyc_lookup.npy')
def ID_Transform(id_value, trans = Location_transform):
    index = trans[id_value-1][-1]
    return index

result = map(ID_Transform, data_uber.loc[:, 'Location_ID'].tolist())

data_uber['Transformed_ID'] = result

In [16]:
data_uber['Transformed_ID'] = result

In [18]:
data = data_uber.groupby('Transformed_ID')['time'].sum().reset_index(level = 1)

In [21]:
data.rename(columns={'Transformed_ID': 'Location_ID_new', 'time': 'Count'}, inplace=True)

In [23]:
data.to_csv('Uber_distribution_2014.csv')

In [29]:
uber_ID_state = np.zeros((265, 6), dtype=object)

In [30]:
def curv_func(x, a, b, c, d):
    return a + b*x + c*np.sin((2*np.pi/7.)*x) + d*np.cos((2*np.pi/7.)*x)

In [31]:
for i in range(265):
    uber_ID_state[i, 0] = i+1
    if i not in uber_ID_list:
        uber_ID_state[i, 1] = 'Uber_not_popuplar'
    else:
        sample  = data_uber[data_uber['Location_ID'] == i+1]
        count_value = sample.groupby(['Month', 'date']).count()['time'].tolist()
        if len(count_value) < 183:
            uber_ID_state[i, 1] = 'Uber_not_popuplar'
        else:
            uber_ID_state[i, 1] = 'popular'
            count_value = np.array(count_value)
            para = optimization.curve_fit(curv_func, range(len(count_value)), count_value, np.zeros(4))[0]
            uber_ID_state[i, 2:] = para

In [32]:
uber_state = pd.DataFrame(uber_ID_state, columns=['Locarion_ID', 'Popularity', 'Uber_a', 'Uber_b', 'Uber_c', 'Uber_d'])

In [34]:
Taxi_ID_state = np.zeros((265, 6), dtype=object)

In [40]:
for i in range(265):
    print i
    Taxi_ID_state[i, 0] = i+1
    sample = data_taxi[data_taxi['Location_ID'] == i+1]
    sample = sample[sample['Year'] == 2014]
    sample = sample[(sample['Month']<=9) & (sample['Month'] >=4)]
    count_value = sample.groupby(['Month', 'Date'])['Count'].sum().tolist()
    if len(count_value) < 183:
        Taxi_ID_state[i, 1] = 'Taxi_not_popuplar'
    else:
        Taxi_ID_state[i, 1] = 'popular'
        count_value = np.array(count_value)
        para = optimization.curve_fit(curv_func, range(len(count_value)), count_value, np.zeros(4))[0]
        Taxi_ID_state[i, 2:] = para

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264


In [41]:
taxi_state = pd.DataFrame(Taxi_ID_state, columns=['Locarion_ID', 'Popularity', 'taxi_a', 'taxi_b', 'taxi_c', 'taxi_d'])

In [43]:
len(taxi_state[taxi_state['Popularity'] == 'popular'])

142

In [134]:
count_value = np.array(count_value)

In [135]:
para = optimization.curve_fit(curv_func, range(len(count_value)), count_value, np.zeros(4))[0]

In [137]:
pred_out = map(lambda x: curv_func(x, para[0], para[1], para[2], para[3]), range(len(count_value)))

In [138]:
pred_linear_out = map(lambda x: para[0]+para[1]*x, range(len(count_value)))

In [101]:
pred_out

[849.09711452820261,
 874.23814533956704,
 1055.1778221561487,
 874.23814533956704,
 557.18666694725721,
 557.18666694725721,
 677.81311815831145,
 788.78388892267526,
 858.73283023070155,
 1092.4465794521132,
 1017.4820561526946,
 797.0272777527158,
 889.31645174094854,
 819.64473735478941,
 647.6565053555488,
 760.74380336591173,
 1017.0550474452026,
 1045.1150977461652,
 759.33151174926206,
 594.8824329507114,
 812.10558415409571,
 685.96492844475051,
 790.90041616867609,
 1251.1958053741139,
 760.74380336591173,
 622.92251850747311,
 609.94077460790209,
 670.27396495762173,
 911.93391134302146,
 1104.7080249415208,
 1308.2651613601738,
 979.99190327169651,
 904.39475814232981,
 870.99427572010666,
 640.11735215485714,
 961.46411412839473,
 844.28714565925964,
 986.91839938663747,
 826.479654926129,
 931.30750132563378,
 617.49989255278433,
 720.93151011645432,
 768.89561365234965,
 1130.5693541630574,
 1032.7659756765295,
 1158.9066313858875,
 834.01880812682259,
 677.8131181583114

In [109]:
plt.plot(range(len(count_value)), count_value, label = 'Real')
plt.plot(range(len(count_value)), pred_out, label = 'Regression')
plt.plot(range(len(count_value)), pred_linear_out, '--', label = 'Linear Part')
plt.legend()
plt.show()

In [44]:
state_all = pd.merge(uber_state, taxi_state, on = ['Locarion_ID'])

In [45]:
state_all.head()

,Locarion_ID,Popularity_x,Uber_a,Uber_b,Uber_c,Uber_d,Popularity_y,taxi_a,taxi_b,taxi_c,taxi_d
0,1,Uber_not_popuplar,0,0,0,0,popular,1842.32,3.90925,263.941,-104.218
1,2,Uber_not_popuplar,0,0,0,0,Taxi_not_popuplar,0,0,0,0
2,3,Uber_not_popuplar,0,0,0,0,Taxi_not_popuplar,0,0,0,0
3,4,popular,62.5616,0.580206,10.893,-20.9385,popular,1176.8,1.66439,-240.548,-529.83
4,5,Uber_not_popuplar,0,0,0,0,Taxi_not_popuplar,0,0,0,0


In [46]:
state_all = state_all.rename(columns={'Locarion_ID': 'Location_ID', 'Popularity_x': 'Uber_Popularity', 'Popularity_y': 'Taxi_Popularity'})

In [47]:
state_all.head()

,Location_ID,Uber_Popularity,Uber_a,Uber_b,Uber_c,Uber_d,Taxi_Popularity,taxi_a,taxi_b,taxi_c,taxi_d
0,1,Uber_not_popuplar,0,0,0,0,popular,1842.32,3.90925,263.941,-104.218
1,2,Uber_not_popuplar,0,0,0,0,Taxi_not_popuplar,0,0,0,0
2,3,Uber_not_popuplar,0,0,0,0,Taxi_not_popuplar,0,0,0,0
3,4,popular,62.5616,0.580206,10.893,-20.9385,popular,1176.8,1.66439,-240.548,-529.83
4,5,Uber_not_popuplar,0,0,0,0,Taxi_not_popuplar,0,0,0,0


In [48]:
Prediction = np.zeros(265, dtype=object)

In [49]:
for i in range(265):
    if state_all.loc[i, 'Uber_Popularity'] == 'Uber_not_popuplar':
        if state_all.loc[i, 'Taxi_Popularity'] == 'Taxi_not_popuplar':
            Prediction[i] = 'Not_Popular'
        else:
            Prediction[i] = 'Taxi_Dominates'
    else:
        if state_all.loc[i, 'Taxi_Popularity'] == 'Taxi_not_popuplar':
            Prediction[i] = 'Uber_Dominates'
        else:
            if state_all.loc[i, 'Uber_b'] - state_all.loc[i, 'taxi_b']<=0:
                Prediction[i] = 'Taxi_Safe'
            else:
                a_u = state_all.loc[i, 'Uber_a']
                b_u = state_all.loc[i, 'Uber_b']
                a_t = state_all.loc[i, 'taxi_a']
                b_t = state_all.loc[i, 'taxi_b']
                pred = (a_t - a_u)/(b_u - b_t)
                Prediction[i] = pred - 183

In [50]:
state_all['Prediction'] = Prediction

In [51]:
state_all

,Location_ID,Uber_Popularity,Uber_a,Uber_b,Uber_c,Uber_d,Taxi_Popularity,taxi_a,taxi_b,taxi_c,taxi_d,Prediction
0,1,Uber_not_popuplar,0,0,0,0,popular,1842.32,3.90925,263.941,-104.218,Taxi_Dominates
1,2,Uber_not_popuplar,0,0,0,0,Taxi_not_popuplar,0,0,0,0,Not_Popular
2,3,Uber_not_popuplar,0,0,0,0,Taxi_not_popuplar,0,0,0,0,Not_Popular
3,4,popular,62.5616,0.580206,10.893,-20.9385,popular,1176.8,1.66439,-240.548,-529.83,Taxi_Safe
4,5,Uber_not_popuplar,0,0,0,0,Taxi_not_popuplar,0,0,0,0,Not_Popular
5,6,Uber_not_popuplar,0,0,0,0,Taxi_not_popuplar,0,0,0,0,Not_Popular
6,7,popular,25.6894,0.871272,-15.5437,-39.4586,popular,1321.01,1.54523,-381.737,-458.315,Taxi_Safe
7,8,popular,6.4124,0.208694,-4.28959,-11.8523,popular,159.2,0.252207,-34.4783,-55.676,Taxi_Safe
8,9,Uber_not_popuplar,0,0,0,0,Taxi_not_popuplar,0,0,0,0,Not_Popular
9,10,Uber_not_popuplar,0,0,0,0,popular,80.5739,0.281526,-0.921691,-2.27517,Taxi_Dominates


In [52]:
state_all.to_csv('Prediction_2014.csv', index=False)

In [139]:
data_taxi_sample = data_taxi[data_taxi['Location_ID'] == 161]

In [140]:
data_taxi_sample = data_taxi_sample[data_taxi_sample['Year'] == 2015]

In [141]:
data_taxi_sample = data_taxi_sample[data_taxi_sample['Month']<=6]

In [142]:
count_value_taxi = data_taxi_sample.groupby(['Month', 'Date'])['Count'].sum().tolist()

In [143]:
count_value_taxi = np.array(count_value_taxi)

In [144]:
para_taxi = optimization.curve_fit(curv_func, range(len(count_value_taxi)), count_value_taxi, np.zeros(4))[0]

In [145]:
para_taxi

array([  2.06570901e+04,  -4.79895614e+00,  -2.79131795e+03,
         2.84683122e+03])

In [146]:
pred_out_taxi = map(lambda x: curv_func(x, para_taxi[0], para_taxi[1], para_taxi[2], para_taxi[3]), range(len(count_value_taxi)))

In [147]:
pred_linear_out_taxi = map(lambda x: para_taxi[0]+para_taxi[1]*x, range(len(count_value_taxi)))

In [150]:
plt.plot(range(len(count_value)), count_value, label = 'Real Uber')
plt.plot(range(len(count_value)), pred_out, label = 'Regression Uber')
plt.plot(range(len(count_value)), pred_linear_out, '--', label = 'Linear Uber')
plt.plot(range(len(count_value_taxi)), count_value_taxi, label = 'Real Taxi')
plt.plot(range(len(count_value_taxi)), pred_out_taxi, label = 'Rregression Taxi')
plt.plot(range(len(count_value_taxi)), pred_linear_out_taxi, '--', label = 'Linear Taxi')
plt.legend()
plt.show()